<a href="https://colab.research.google.com/github/AnaAretusa/BootcampCD/blob/Modulo2/DesafioM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 51 kB/s 
     |████████████████████████████████| 199 kB 71.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=3812b8fb24508453ad6d4ab745539c53e4dbac78d8067115b86c3cf61d303f6c
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [9]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import *

In [10]:
spark = SparkSession.builder.getOrCreate()

Importando a base

In [11]:
df = (spark.read
      .format("csv")
      .option("header", "True")
      .option("inferSchema", "True")
      .load("/content/stroke_data.csv")
)

In [8]:
df.show(5)

+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  0|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level|  bmi| smoking_status|stroke|
+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  1|Female|18.0|           0|            0|          No|      Private|         Urban|            94.19|12.12|         smokes|     1|
|  2|  Male|58.0|           1|            0|         Yes|      Private|         Rural|           154.24| 33.7|   never_smoked|     0|
|  3|Female|36.0|           0|            0|         Yes|     Govt_job|         Urban|            72.63| 24.7|         smokes|     0|
|  4|Female|62.0|           0|            0|         Yes|Self-employed|         Rural|            85.52| 31.2|formerly smoked|     0|
|  5|Female|82.0|           0|            0|         Yes|     

##Pergunta 1
Quantos registros existem no arquivo?

In [9]:
df.count()

67135

##Pergunta 2
Quantas colunas existem no arquivo? Quantas são numéricas? Ao ler o arquivo com spark.read.csv, habilite inferSchema=True. Use a função printSchema() da API de Dataframes.

In [10]:
  df.printSchema()

root
 |-- 0: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [13]:
df.describe

<bound method DataFrame.describe of DataFrame[0: int, gender: string, age: double, hypertension: int, heart_disease: int, ever_married: string, work_type: string, Residence_type: string, avg_glucose_level: double, bmi: double, smoking_status: string, stroke: int]>

##Pergunta 3
No conjunto de dados, quantos pacientes sofreram e não sofreram derrame (stroke), respectivamente?

In [14]:
df.groupby("stroke").count().show()

+------+-----+
|stroke|count|
+------+-----+
|     1|40287|
|     0|26848|
+------+-----+



##Pergunta 4
A partir do dataframe, crie uma tabela temporária usando df.createOrReplaceTempView('table') e a seguir use spark.sql para escrever uma consulta SQL que obtenha quantos pacientes tiveram derrame por tipo de trabalho (work_type). Quantos pacientes sofreram derrame e trabalhavam respectivamente, no setor privado, de forma independente, no governo e quantas são crianças?

In [121]:
df.createTempView('temp')

In [22]:
spark.sql("SELECT work_type, count(*) FROM temp WHERE stroke =1 GROUP BY work_type").show()

+-------------+--------+
|    work_type|count(1)|
+-------------+--------+
| Never_worked|      85|
|Self-employed|   10807|
|      Private|   23711|
|     children|     520|
|     Govt_job|    5164|
+-------------+--------+



##Pergunta 5
Escreva uma consulta com spark.sql para determinar a proporção, por gênero, de participantes do estudo. A maioria dos participantes é:

In [25]:
spark.sql("SELECT gender, count(*) FROM temp GROUP BY gender").show()

+------+--------+
|gender|count(1)|
+------+--------+
|Female|   39530|
| Other|      11|
|  Male|   27594|
+------+--------+



##Pergunta 6
Escreva uma consulta com spark.sql para determinar quem tem mais probabilidade de sofrer derrame: hipertensos ou não-hipertensos. Você pode escrever uma consulta para cada grupo. A partir das probabilidades que você obteve, você conclui que:

In [129]:
spark.sql("SELECT hypertension, count(*) FROM temp WHERE stroke =1 GROUP BY hypertension").show()

+------------+--------+
|hypertension|count(1)|
+------------+--------+
|           1|    8817|
|           0|   31470|
+------------+--------+



##Pergunta 7
Escreva uma consulta com spark.sql que determine o número de pessoas que sofreram derrame por idade. Com qual idade o maior número de pessoas do conjunto de dados sofreu derrame?

In [42]:
spark.sql("SELECT age, count(*) FROM temp GROUP BY age ORDER BY count(*) DESC").show()

+----+--------+
| age|count(1)|
+----+--------+
|79.0|    3258|
|78.0|    2672|
|80.0|    2141|
|81.0|    2005|
|82.0|    1657|
|63.0|    1294|
|66.0|    1195|
|77.0|    1190|
|74.0|    1184|
|57.0|    1160|
|70.0|    1151|
|76.0|    1088|
|67.0|    1070|
|51.0|    1067|
|65.0|    1046|
|75.0|    1015|
|52.0|    1003|
|58.0|     999|
|59.0|     994|
|61.0|     988|
+----+--------+
only showing top 20 rows



##Pergunta 8
Usando a API de dataframes, determine quantas pessoas sofreram derrames após os 50 anos.

In [53]:
df.filter(df.age>50).filter(df.stroke ==1).count()

28938

##Pergunta 9
Usando spark.sql, determine qual o nível médio de glicose para pessoas que, respectivamente, sofreram e não sofreram derrame.

In [7]:
derrame = df.filter(df.stroke == 1)

In [9]:
derrame.show(2)

+---+------+----+------------+-------------+------------+---------+--------------+-----------------+-----+--------------+------+
|  0|gender| age|hypertension|heart_disease|ever_married|work_type|Residence_type|avg_glucose_level|  bmi|smoking_status|stroke|
+---+------+----+------------+-------------+------------+---------+--------------+-----------------+-----+--------------+------+
|  1|Female|18.0|           0|            0|          No|  Private|         Urban|            94.19|12.12|        smokes|     1|
|  5|Female|82.0|           0|            0|         Yes|  Private|         Rural|            59.32| 33.2|        smokes|     1|
+---+------+----+------------+-------------+------------+---------+--------------+-----------------+-----+--------------+------+
only showing top 2 rows



In [17]:
derrame.agg({"avg_glucose_level":"avg"}).show()

+----------------------+
|avg(avg_glucose_level)|
+----------------------+
|    119.95307046938272|
+----------------------+



In [18]:
noderrame = df.filter(df.stroke == 0)

In [19]:
noderrame.agg({"avg_glucose_level":"avg"}).show()

+----------------------+
|avg(avg_glucose_level)|
+----------------------+
|    103.60273130214506|
+----------------------+



##Pergunta 10
Qual é o BMI (IMC = índice de massa corpórea) médio de quem sofreu e não sofreu derrame?

In [21]:
derrame.agg({"bmi":"avg"}).show()

+------------------+
|          avg(bmi)|
+------------------+
|29.942490629729495|
+------------------+



In [20]:
noderrame.agg({"bmi":"avg"}).show()

+------------------+
|          avg(bmi)|
+------------------+
|27.989678933253657|
+------------------+



##Pergunta 11
Crie um modelo de árvore de decisão que prevê a chance de derrame (stroke) a partir das variáveis contínuas/categóricas: idade, BMI, hipertensão, doença do coração, nível médio de glicose. Use o conteúdo da segunda aula interativa para criar e avaliar o modelo.

 

Qual a acurácia de um modelo construído?

In [2]:
from pyspark.ml.feature import VectorAssembler

In [15]:
assembler = VectorAssembler(inputCols=['age','bmi','hypertension','heart_disease','avg_glucose_level'], outputCol='features')

In [16]:
from pyspark.ml.classification import DecisionTreeClassifier

In [17]:
classifier = DecisionTreeClassifier(labelCol="stroke", featuresCol="features")

In [18]:
from pyspark.ml import Pipeline

In [19]:
pipeline = Pipeline(stages=[assembler, classifier])

In [20]:
train_data, test_data = df.randomSplit([0.7,0.3])

In [21]:
predictSurvivedModel = pipeline.fit(train_data)

In [22]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [26]:
predictions = predictSurvivedModel.transform(test_data)

In [27]:
evaluetor = MulticlassClassificationEvaluator(labelCol='stroke', predictionCol='prediction', metricName='accuracy')
accuracy = evaluetor.evaluate(predictions)
accuracy

0.6872936887224713

##Pergunta 12
Adicione ao modelo as variáveis categóricas: gênero e status de fumante. Use o conteúdo da aula interativa para lidar com as variáveis categóricas.  A acurácia (qualidade) do modelo aumentou para:

In [29]:
from pyspark.ml.feature import StringIndexer,OneHotEncoder

In [85]:
gender_indexer = StringIndexer(inputCol='gender', outputCol='gender_indexer')
gender_encoder = OneHotEncoder(inputCol='gender_indexer', outputCol='gender_vector')

In [87]:
smoking_indexer = StringIndexer(inputCol='smoking_status', outputCol='smoking_indexer')
smoking_encoder = OneHotEncoder(inputCol='smoking_indexer', outputCol='smoking_vector')

In [88]:
assembler12 = VectorAssembler(inputCols=['age','bmi','hypertension','heart_disease','avg_glucose_level','gender_vector','smoking_vector'], outputCol='features')

In [89]:
classifier12 = DecisionTreeClassifier(labelCol="stroke", featuresCol="features")

In [90]:
pipeline12 = Pipeline(stages=[gender_indexer,gender_encoder,smoking_indexer, smoking_encoder,assembler12, classifier12])

In [91]:
train_data12, test_data12 = df.randomSplit([0.7,0.3])

In [92]:
predictSurvivedModel12 = pipeline12.fit(train_data12)

In [93]:
predictions12 = predictSurvivedModel12.transform(test_data12)

In [94]:
evaluetor12 = MulticlassClassificationEvaluator(labelCol='stroke', predictionCol='prediction12', metricName='accuracy12')
accuracy12 = evaluetor.evaluate(predictions12)
accuracy12

0.8378005518328735

##Pergunta 13
Qual dessas variáveis é mais importante no modelo de árvore de decisão que você construiu na questão (12)?


In [144]:
decisionTreeModel12 = predictSurvivedModel12.stages[5]

In [145]:
list(zip(assembler12.getInputCols(), decisionTreeModel12.featureImportances))

[('age', 0.1688569746903108),
 ('bmi', 0.00016147567907665708),
 ('hypertension', 0.0),
 ('heart_disease', 0.0),
 ('avg_glucose_level', 0.00843132209651314),
 ('gender_vector', 0.0),
 ('smoking_vector', 0.0)]

##Pergunta 14
Qual a profundidade da árvore de decisão da questão (12)?

In [108]:
decisionTreeModel12.depth

5

##Pergunta 15
Quantos nodos a árvore de decisão possui?

In [109]:
assembler12.getInputCols()

['age',
 'bmi',
 'hypertension',
 'heart_disease',
 'avg_glucose_level',
 'gender_vector',
 'smoking_vector']

##Salvando a base

In [ ]:
basePath = "C:/Users/ana.santos/Documents/MBA/Ciencias_Dados/Mod 2"
decisionTreeModel12.write().overwrite().save(basePath + "/model")